In [141]:
from neo4j import GraphDatabase
import re

In [73]:
class neo4j_connection:
    
    
    def __init__(self, uri, user, password):
        self._driver = GraphDatabase.driver(uri, auth=(user, password))
        
        
    def close(self):
        self._driver.close()

        
    def run(self, string):
        with self._driver.session() as session:
            session.run(string) 
        
        
    def nuke_it(self):
        nuke_string = """
            MATCH (n)
            DETACH DELETE n
            """
        with self._driver.session() as session:
            session.run(nuke_string)

In [74]:
uri = "bolt://localhost:7687"
user = "neo4j"
password = "47westrange"

In [75]:
example = neo4j_connection(uri, user, password)

In [35]:
example.run("""
    CREATE (john:Person {name: 'John'})
    CREATE (joe:Person {name: 'Joe'})
    CREATE (steve:Person {name: 'Steve'})
    CREATE (sara:Person {name: 'Sara'})
    CREATE (maria:Person {name: 'Maria'})
    CREATE (john)-[:FRIEND]->(joe)-[:FRIEND]->(steve)
    CREATE (john)-[:FRIEND]->(sara)-[:FRIEND]->(maria)
""")

In [36]:
example.nuke_it()

In [37]:
example.close()

In [76]:
import json

with open('/Users/mattkirby/repos/Science-NLP/documents.json') as json_file:
    data = dict(json.load(json_file))

In [77]:
import pandas as pd

df = pd.DataFrame(data).T

In [78]:
df.columns

Index(['contents', 'emails', 'filename', 'institutions', 'people', 'places'], dtype='object')

In [101]:
df.index = df.index.str.replace('-', '_')
df.index = df.index.str.replace(' ', '_')
df.index = df.index.str.replace('(', '_')
df.index = df.index.str.replace(')', '_')
df.index = df.index.str.replace('90', 'Ninety')
df.index = df.index.str.replace('.', '_')
df.index = df.index.str.replace('?', '_')

In [110]:
df['people'] = df['people'].apply(lambda x: list(set(x))).tolist()

In [146]:
df.index

Index(['Navigation_to_Small_Bodies', 'ASTRONOMICAL_ENGINEERING',
       'Phase_II_of_the_Main_Belt_Asteroid_Spectrosopic_Survey',
       'Devlopment_of_Xenon_Hall_Thrusters',
       'Mine_planning_for_Asteroid_Ore_Bodies', 'CSR_in_space',
       'Interplanetary_Trajectory', 'SmallSat_Constelation',
       'Directed_Passpermia', 'The_Rise_of_Interplanetary_Cubesats',
       'Launch_and_Deployment_of_Distributted_Small_Satelite_Systems',
       'Options_and_Uncertanties_in_Planetary_Defence',
       'The_Compositional_Structure_of_The_Asteroid_Belt',
       'Vacuum_induction_melting_of_NiTi_shape_memory', 'Project_NERVA',
       'NEO_Characterization',
       'Photometric_survey_and_taxonomic_identifications_of_92_near_Earth_asteroids',
       'Organic_matter_preserved_in_3_billion_year_old_mudstones',
       'Defining_a_Successful_Asteroid_Mining_Program', 'Novum',
       'New_Venice_Project', 'Asteroid_Retrieval_Feasibility',
       'Meterorite_as_raw_Meterial_for_Direct_Metal_Printing

In [144]:
example.nuke_it()

In [145]:
for i in df.index.tolist():
    
    # Paper CREATE statement
    paper_statement = "CREATE ({papername}:Paper {{name: '{papername}'}})"\
        .format(papername = str(i))
    
    # Execute Paper CREATE statement
    #print(paper_statement)
    example.run(paper_statement)
    
    for person in df.at[i, 'people']:

        person = re.sub('[^a-zA-Z]+', '_', person)

        # Person CREATE statement
        person_statement = "CREATE ({person}:Person {{name: '{person}'}})"\
        .format(person = str(person))
        
        # Execute Person CREATE statement
        #print(person_statement)
        example.run(person_statement)
        
        # Author connection statement
        author_connection = "CREATE ({person})-[:WROTE]->({papername})"\
        .format(person = str(person), papername = str(i))
        
        # Execute Author connection statement
        #print(author_connection)
        example.run(author_connection)
        
    #print('\n\n')
    


KeyboardInterrupt: decoding with 'utf-8' codec failed (KeyboardInterrupt: )